To run below code, you need to following python packages installed:

- tweepy
- kafka-python

install them with a package manager of your choice. 
We used pip3 and anaconda.

To run this code, you also need twitter developer access. See https://apps.twitter.com. After creating an app, add the consumer key, consumer secret, access token, and access token secret in a .json file named "credentials.json". The json structure we use is:

```bash
{ 
    "Credentials":
       { 
          "ACCESS_TOKEN":"",
          "ACCESS_TOKEN_SECRET":"",
          "CONSUMER_KEY":"",
          "CONSUMER_SECRET":""
       }
    
 }
```
 Now you can run the code in this notebook.

First we need to import you credentials for the Tweepy authentication

In [1]:
import json
with open('credentials.json') as f:
    data = json.load(f)
    cred = data["Credentials"]
    access_token = cred['ACCESS_TOKEN']
    access_token_secret = cred['ACCESS_TOKEN_SECRET']
    consumer_key = cred['CONSUMER_KEY']
    consumer_secret = cred['CONSUMER_SECRET']


Now we can start up the kafka broker. For this we need a kafka server and a zookeeper running. We started them with the following bash commands:

```bash
    $KAFKA_HOME/bin/zookeeper-server-start.sh $KAFKA_HOME/config/zookeeper.properties
    $KAFKA_HOME/bin/kafka-server-start.sh $KAFKA_HOME/config/server.properties 
```

Create a topic called "tweets"
```bash
    $KAFKA_HOME/bin/kafka-topics.sh --create --zookeeper localhost:2181 --replication-factor 1 --partitions 1 --topic tweets
```

Now we define the kafka configuration, connection procedure, and publish function.

In [2]:
import uuid
from kafka import KafkaProducer

#Kafka conf
server = 'localhost:9092'
topic_name = "tweets"

def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        #print('Message published successfully.')
    except Exception as ex:
        print('Exception in publishing message')
        print(str(ex))


def connect_kafka_producer(server):
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[server], api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _producer
kafka_producer = connect_kafka_producer(server)

Now run the producer. This producer connects to Twitter through Tweepy and posts the tweets as messages to Kafka. The following cell will run indefinetly, interrupt the kernel to stop. 

In [4]:
import sys
from tweepy import OAuthHandler, API, Stream
from tweepy.streaming import StreamListener

# Replace the "None"s by your own credentials
ACCESS_TOKEN = access_token
ACCESS_TOKEN_SECRET = access_token_secret
CONSUMER_KEY = consumer_key
CONSUMER_SECRET = consumer_secret

auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = API(auth, wait_on_rate_limit=True,
          wait_on_rate_limit_notify=True)

class Listener(StreamListener):
    def __init__(self, output_file=sys.stdout):
        self.output_file = output_file
        super(Listener,self).__init__()
    def on_status(self, status):
        hashtags = status.entities['hashtags']
        
        if len(hashtags) > 0:
            publish_message(kafka_producer, topic_name, str(uuid.uuid4()), json.dumps(status._json))
        #print(json.dumps(status._json), file=self.output_file)
    def on_error(self, status_code):
        print(status_code)
        return False

#For debugging the tweepy output to a file, uncomment the above print statement.
output = open('stream_output.json', 'w+')
listener = Listener(output_file=output)
stream = Stream(auth=api.auth, listener=listener, tweet_mode= 'extended')

try:
    print('Start streaming.')
    stream.filter(track=['Hong Kong'])
except KeyboardInterrupt:
    print("Stopped.")
finally:
    print('Done.')
    stream.disconnect()
    output.close()

Start streaming.
Stopped.
Done.
